In [3]:
import edgedb
import json
import requests
from bs4 import BeautifulSoup
from decimal import Decimal
import re

f=open('bike_info_img_url.json')
data=json.load(f)
print(len(data))

print(data[0])

client=edgedb.create_client('crawl')

63756
{'Brand': 'Specialized', 'Model': 'Kenevo Expert ', 'Category': 'E-bike', 'Sub-Category': 'Enduro', 'Price': '13,000', 'Currency': 'AUD', 'Year': '2022', 'img_url': 'https://bikes.fan/wp-content/uploads/bike_images/specialized-kenevo-expert-2022.jpeg', 'Frame': 'Aluminum', 'Wheels': '27.5"', 'Travel Front': '180mm', 'Travel Rear': '180mm', 'Fork': 'RockShox BoXXer Select', 'Groupset': 'X01 Eagle, SRAM', 'Suspension': 'Full', 'Motor': 'Specialized mid-drive', 'Battery': 'Specialized M2-700, integrated battery w/rock guard, 700Wh', 'Drivetrain': '1 × 12', 'Brakes': 'Hydraulic Disc'}


In [4]:

for bike in data: 
    bike['Travel Front']=bike.setdefault('Travel Front')
    bike['Travel Rear']=bike.setdefault('Travel Rear')
    bike['Motor']=bike.setdefault('Motor')
    bike['Battery']=bike.setdefault('Battery')
    bike['Drivetrain']=bike.setdefault('Drivetrain')
    bike['Fork']=bike.setdefault('Fork')
    bike['Price']=bike.setdefault('Price','')
    bike['Currency']=bike.setdefault('Currency')
    bike['Year']=bike.setdefault('Year')
    bike['Groupset']=bike.setdefault('Groupset')
    bike['Suspension']=bike.setdefault('Suspension')
    bike['Brakes']=bike.setdefault('Brakes')
    bike['Frame']=bike.setdefault('Frame')
    bike['Wheels']=bike.setdefault('Wheels')
    bike['Brand']=bike.setdefault('Brand')
    bike['Model']=bike.setdefault('Model')
    bike['Category']=bike.setdefault('Category')
    bike['Sub-Category']=bike.setdefault('Sub-Category')
    bike['img_url']=bike.setdefault('img_url')
    
    #insert brand
    client.query('''
        INSERT Brands {            
                brand := <str>$brand
            }
            unless conflict on (.brand)
        ''',brand=bike['Brand'] if bike['Brand']!=None else "")
    
    #update brand
    client.query('''
        Update Brands 
        FILTER .brand = <str>$brand
        SET {            
                categories += (
                    INSERT Categories
                    {
                      category:= <str>$category
                    }                    
                    unless conflict on (.category) else Categories
                )
            }
        ''',brand=bike['Brand'].strip() if bike['Brand']!=None else "", category=bike['Category'].strip() if bike['Category']!=None else "")
    
    #update category
    client.query('''
        Update Categories 
        FILTER .category = <str>$category
        SET {            
                subcategories += (
                    INSERT Subcategories
                    {
                      subcategory:= <str>$subcategory
                    }
                    unless conflict on (.subcategory) else Subcategories
                )
            }
        ''',category=bike['Category'].strip() if bike['Category']!=None else "", subcategory=bike['Sub-Category'].strip() if bike['Sub-Category']!=None else "")
   
     #update subcategory
    client.query('''
        Update Subcategories 
        FILTER .subcategory = <str>$subcategory
        SET {            
                models += (
                    INSERT Models
                    {
                      model:= <str>$model,
                      year := <str>$year
                    }
                    unless conflict on (.model, .year) else Models
                )
            }
        ''',subcategory=bike['Sub-Category'].strip() if bike['Sub-Category']!=None else "", model=bike['Model'].strip() if bike['Model']!=None else "", year=bike['Year'].strip() if bike['Year']!=None else "")
 
     #update category
    client.query('''
        Update Categories 
        FILTER .category = <str>$category
        SET {            
                models += (
                    SELECT Models
                    filter
                      .model= <str>$model
                      and
                      .year = <str>$year
                )
            }
        ''',category=bike['Category'].strip() if bike['Category']!=None else "", model=bike['Model'].strip() if bike['Model']!=None else "", year=bike['Year'].strip() if bike['Year']!=None else "")
  
    #update brand
    client.query('''
        Update Brands 
        FILTER .brand = <str>$brand
        SET {            
                models += (
                    SELECT Models
                    filter
                      .model= <str>$model
                      and
                      .year = <str>$year
                )
            }
        ''',brand=bike['Brand'].strip() if bike['Brand']!=None else "", model=bike['Model'].strip() if bike['Model']!=None else "", year=bike['Year'].strip() if bike['Year']!=None else "")
 
    
    #udpate Model
    client.query('''
        Update Models 
        FILTER .model = <str>$model 
                and 
                .year = <str> $year
        SET { 
                bikes += (
                          INSERT Bikes {
                            frame := <str>$frame,
                            wheels := <str>$wheels,
                            travelfront := <str>$travelfront,
                            travelrear := <str>$travelrear,
                            fork := <str>$fork,
                            groupset := <str>$groupset,
                            suspension := <str>$suspension,
                            motor := <str>$motor,
                            battery := <str>$battery,
                            drivetrain := <str>$drivetrain,
                            brakes := <str>$brakes,
                            url := <str>$url,
                            price :=  <decimal>$price,
                            currency := <str>$currency
                        }
                        unless conflict on (.url) else Bikes
                      )           
            }
        '''#,brand=bike['Brand'] if bike['Brand']!=None else ""
           , model=bike['Model'].strip() if bike['Model']!=None else ""
           #, category=bike['Category'] if bike['Category']!=None else ""
           #, subcategory=bike['Sub-Category'] if bike['Sub-Category']!=None else ""
           , price=Decimal(bike['Price'].strip().replace(',','')) if bike['Price']!=None and len(bike['Price'])>0 else 0
           , currency=bike['Currency'].strip() if bike['Currency']!=None else ""
           , year=bike['Year'].strip() if bike['Year']!=None else ""
           , frame=bike['Frame'].strip() if bike['Frame']!=None else ""
           , wheels=bike['Wheels'].strip() if bike['Wheels']!=None else ""
           , travelfront=bike['Travel Front'].strip() if bike['Travel Front']!=None else ""
           , travelrear=bike['Travel Rear'].strip() if bike['Travel Rear']!=None else ""
           , fork=bike['Fork'].strip() if bike['Fork']!=None else ""
           , groupset=bike['Groupset'].strip() if bike['Groupset']!=None else ""
           , suspension=bike['Suspension'].strip() if bike['Suspension']!=None else ""
           , motor=bike['Motor'].strip() if bike['Motor']!=None else ""
           , battery=bike['Battery'].strip() if bike['Battery']!=None else ""
           , drivetrain=bike['Drivetrain'].strip() if bike['Drivetrain']!=None else ""
           , brakes=bike['Brakes'].strip() if bike['Brakes']!=None else ""
           , url=bike['img_url'].strip() if bike['img_url']!=None else "")
        
 

client.close()